In [195]:
from google.colab import drive 
drive.mount('/content/gdrive')
path = './gdrive/My Drive/ds/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
import cv2
import numpy as np

X = None
Y = np.array([])

def read_files(X, Y, path, ans):
  files = os.listdir(path)
  for name in files:
    img = cv2.imread(path + '/' + name, 0)
    if img.shape != 0:
      img = cv2.resize(img, (256, 256))
      vect = img.reshape(1, 256 ** 2)
      vect = vect / 255.
      X = vect if (X is None) else np.vstack((X, vect)) 
      Y = np.append(Y, ans)
  return X, Y
X_0,Y_0=read_files(X,Y,path+'logloss_1',1);
X_1,Y_1=read_files(X,Y,path+'logloss_0',0);
X = np.concatenate([X_0, X_1])
Y = np.concatenate([Y_0, Y_1])


In [0]:
from sklearn.model_selection import train_test_split


X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.3,
                                                    random_state=48)

In [0]:
def init_model(input_size=256):
    ###ЗАДАЧА: проинициализируйте веса модели так, чтобы массив w имел размер (input_size^2,1), а b был числом
    w = np.array([0] * (input_size ** 2))
    b = 0
    return w, b

def sigmoid(z):
  ###ВАЖНО: функция принимает на вход массив любых размеров, на выход возвращает массив такого же размера
  return 1.0/ (1 + np.exp(-z))
   

def propagate(w, b, X, Y):
    w, X, Y = np.array(w), np.array(X), np.array(Y)
    """
    Подсчет текущего предсказания (оно же forward propagation) и градиента функции ошибки (оно же backward propagation)

    Input:
    w -- веса, numpy_array размера (num_px * num_px * 3, 1)
    b -- смещение, скалярная величина
    X -- данные размера (num_px * num_px * 3, кол-во образцов)
    Y -- вектор истинных ответов размера (1, кол-во образцов)

    Return:
    cost -- текущая функция потерь
    dw -- градиент функции ошибки по w
    db -- градиент функции ошибки по b (по сути производная по b)
    
    """
    
    m = X.shape[1]
          
    
    A = sigmoid(np.dot(w.T,X.T) + b)
    
    cost = -1.0 / m * np.sum(Y * np.log(A) + (1.0 - Y) * np.log(1.0 - A))
      
    dw = 1 / m *(np.dot(X.T,(A - Y).T))
    db = 1 / m *(np.sum(A - Y))
        
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    Оптимизация с помощью простого градиентного спуска
    
    Input:
    w -- веса, numpy_array размера (num_px * num_px * 3, 1)
    b -- смещение, скалярная величина
    X -- данные размера (num_px * num_px * 3, кол-во образцов)
    Y -- вектор истинных ответов размера (1, кол-во образцов)
    num_iterations -- кол-во итераций алгоритма оптимизации
    learning_rate -- коэффициент learning rate
    print_cost -- True, если хотите выводить функцию ошибки на каждых 100 итерациях
    
    Returns:
    params -- словарь, содержащий w и b
    grads -- словарь, содержащий градиенты функции ошибки по w и b соответственно
    costs -- массив (list) со значением функции ошибки для каждой итерации (так делают для визуализации)
    
    Подсказка:
    
        1) Используйте ранее написанную функцию propagate().
        2) Обновляйте параметры w и b согласно формуле 10.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        ###Напишите значения для градиентов и функции ошибки
        grads, cost = propagate(w, b, X, Y)
        
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # обновление параметров
        
        ### START CODE HERE ###
        w = w-learning_rate*dw
        b = b-learning_rate*db
        ### END CODE HERE ###
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs
  

def predict(w, b, X):
    '''
    
    Inputs:
    w
    b
    X -- данные размера (num_px * num_px * 3, кол-во образцов)
    
    Returns:
    Y_prediction
    '''
    m = X.shape[0]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.T.shape[0], 1)
    
    
    ### START CODE HERE ### (≈ 1 line of code)
    A = sigmoid(np.dot(w.T,X.T) + b)
    
    ### END CODE HERE ###
    
    for i in range(A.shape[1]):
        
        # Установите порог, выше которого считаем, что модель выдает 1, а ниже - ноль
        ### START CODE HERE ###
        if(A[:, i] <= 0.5):
            Y_prediction[:, i] = 0
        else:
            Y_prediction[:, i] = 1
        ### END CODE HERE ###
    
    return Y_prediction

w,b=init_model()
 

parameters, grads, costs= optimize(w, b, X_train, Y_train, 10000, 0.005, print_cost = True)

w = parameters["w"]
b = parameters["b"]


y_train = predict(w,b,np.array(X_train))
y_test = predict(w, b, X_test)


from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(Y_train,y_train[0])
test_accuracy = accuracy_score(Y_test,y_test[0])
print(train_accuracy)
print(test_accuracy)


Cost after iteration 0: 0.000476
Cost after iteration 100: 0.000464
Cost after iteration 200: 0.000456
Cost after iteration 300: 0.000450
Cost after iteration 400: 0.000446
Cost after iteration 500: 0.000442
Cost after iteration 600: 0.000438
Cost after iteration 700: 0.000434
Cost after iteration 800: 0.000431
Cost after iteration 900: 0.000428
Cost after iteration 1000: 0.000425
Cost after iteration 1100: 0.000422
Cost after iteration 1200: 0.000418
Cost after iteration 1300: 0.000416
Cost after iteration 1400: 0.000413
Cost after iteration 1500: 0.000410
Cost after iteration 1600: 0.000407
Cost after iteration 1700: 0.000404
Cost after iteration 1800: 0.000402


In [199]:
},


SyntaxError: ignored